In [2]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
using QuantumDots, QuantumDots.BlockDiagonals, LinearAlgebra
using Plots
#using Symbolics
using Latexify
#using LinearSolve # Solving for stationary state
#using OrdinaryDiffEq # For time evolution
#using SymPy # For analytic solutions

  Activating project at `c:\Users\svens\.julia\dev\QuantumDots\examples`


### Defining a basis
To specify a basis, define the indices and optionally a conserved quantum number


In [9]:
N=2
d = FermionBasis(1:N, (:↑, :↓); qn = QuantumDots.parity)

FermionBasis{4,Tuple{Int64, Symbol},SparseArrays.SparseMatrixCSC{Int64, Int64},QuantumDots.AbelianFockSymmetry{Vector{Int64}, Dictionaries.Dictionary{Int64, Int64}, Int64, typeof(QuantumDots.parity)}}:
keys = {(1, :↑), (2, :↑), (1, :↓), (2, :↓)}

The basis allows easy access to the fermionic operators in the many body basis, as well as dictionaries that define how the basis is ordered

In [10]:
d[1,:↑] #is a SparseArray representation of the annihilation operator

16×16 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
⎡⠀⠀⠀⠀⠐⠄⠀⠀⎤
⎢⠀⠀⠀⠀⠀⠀⠁⢀⎥
⎢⠁⢀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠐⠄⠀⠀⠀⠀⎦

In [11]:
Matrix(d[N,:↓]') # Creation operator. ' is hermitian conjugate. Use Matrix to convert to dense matrix.

16×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0

In [12]:
d[1,:↑]'d[1,:↑] # Number operator 

16×16 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
⎡⠁⢀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠐⠄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠐⠄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠁⢀⎦

In [13]:
sum(f -> f'f, d) |> Matrix # Total number operator

16×16 Matrix{Int64}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4

In [14]:
b = QuantumDots.FermionBdGBasis(collect(Base.product(1:N, (:↑, :↓))))

QuantumDots.FermionBdGBasis{4, Tuple{Int64, Symbol}}({(1, :↑) = 1, (2, :↑) = 2, (1, :↓) = 3, (2, :↓) = 4})

In [25]:
sum(f -> f'f, b)

8×8 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
 1  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  1  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  1  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  1   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅  -1   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅  -1   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅   ⋅  -1   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅  -1

In [66]:
[b for b in b]


In [2]:
N = 2
c = FermionBasis(1:N, qn = QuantumDots.parity)

FermionBasis{4,Tuple{Int64, Symbol},SparseArrays.SparseMatrixCSC{Int64, Int64},QuantumDots.NoSymmetry}:
keys = {(1, :↑), (2, :↑), (1, :↓), (2, :↓)}

\uparrow

In [ ]:
↑,↓

In [ ]:
d = FermionBasis(1:N, (:↑,:↓))

### Kitaev chain hamiltonian
We'll implement the interacting Kitaev chain hamiltonian
$$
\begin{equation}
H = \sum_{n = 1}^N \mu c_n^\dagger c_n + \sum_{n=1}^{N-1} (t c_{n+1}^\dagger c_n + \Delta c_{n+1}c_n + \mathrm{h.c}) + V c_{n+1}^\dagger c_{n+1} c_n^\dagger c_n,
\end{equation}
$$
and calculate the energy gap and majorana polarization. We'll use these to tune for a sweet spot where the system has two separated majoranas.

In [12]:
ham(μ,t,Δ,V, c = c) = Hermitian(sum(μ*c[n]'c[n] for n in 1:N) + sum(t*c[n+1]'c[n] + Δ*c[n+1]c[n] + hc + V*c[n]'c[n]c[n+1]'c[n+1] for n in 1:N-1))

ham (generic function with 2 methods)

### Majorana polarization

$$\begin{align*}
w_n &= \lang \psi_o | (c_n^\dagger + c_n)  | \psi_e \rang \\
z_n &= \lang \psi_o | (c_n^\dagger - c_n)  | \psi_e \rang \\
MP_R &= \left| \sum_{n \in R} w_n^2 - z_n^2 \right|
\end{align*}$$



In [110]:
function MP(ψₒ, ψₑ, c, R)
    w = [ψₒ' * (c[n]' + c[n])ψₑ for n in R]
    z = [ψₒ' * (c[n]' - c[n])ψₑ for n in R]
    abs(sum(w .^ 2 - z .^ 2))
end

MP (generic function with 1 method)

In [111]:
MP(vo,ve,c,[1])

|                                             2                               
|/                  2*Δ                      \    /                  2*Δ      
||--------------------------------------- - 1|  - |---------------------------
||             __________________________    |    |             ______________
||            /  2              2      2     |    |            /  2           
|\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 4*μ      /    \V + 2*μ - \/  V  + 4*V*μ + 

                 2|
                \ |
------------ + 1| |
____________    | |
   2      2     | |
4*Δ  + 4*μ      / |

In [112]:
LD(ψₒ, ψₑ, c, R) = norm(reduced_density_matrix(ψₒψₒ' - ψₑψₑ', R, c))

ErrorException: cannot define function LD; it already has a value

### Using SymPy for analytical solutions

In [91]:
using SymPy
@syms μ::real t::real Δ::real V::real
H = blockdiagonal(ham(μ, t, Δ, V), c)


4×4 BlockDiagonal{Sym, Hermitian{Sym, SparseArrays.SparseMatrixCSC{Sym, Int64}}}:
 μ  t   0        0
 t  μ   0        0
 0  0   0       -Δ
 0  0  -Δ  V + 2*μ

In [92]:
Mo, Me = sympy.Matrix.(Matrix.(blocks(H)))

2-element Vector{Matrix{Sym}}:
 [μ t; t μ]
 [0 -Δ; -Δ V + 2*μ]

In [93]:
vecs_o, vals_o = Mo.diagonalize()
vecs_e, vals_e = Me.diagonalize()

(Sym[-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) -2*Δ/(V + 2*μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)); 1 1], Sym[V/2 + μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2 0; 0 V/2 + μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2])

In [63]:
Ee = diag(vals_e)[1]
ve = vcat(zero(vecs_e[:, 1]), vecs_e[:, 1])

4-element Vector{Sym}:
                                                  0
                                                  0
 -2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))
                                                  1

In [65]:
Eo = diag(vals_o)[1]
vo = vcat(vecs_o[:, 1], zero(vecs_o[:, 1]))

4-element Vector{Sym}:
 -1
  1
  0
  0

In [94]:
mp = majorana_polarization(majorana_coefficients(vo, ve, c)..., (1,))

(mp = Abs(-(-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2 + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2)/((-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)*(-2*Δ/(V + 2*μ - conjugate(sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))) - 1) + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)*(2*Δ/(V + 2*μ - conjugate(sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))) - 1)), mpu = Abs(-(-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2 + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2))

In [98]:
latexify(mp.mpu)

L"$\mathrm{Abs}\left(  - \left( \frac{-2 \cdot \Delta}{V + 2 \cdot \mu - \sqrt{V^{2} + 4 \cdot V \cdot \mu + 4 \cdot \Delta^{2} + 4 \cdot \mu^{2}}} - 1 \right)^{2} + \left( \frac{2 \cdot \Delta}{V + 2 \cdot \mu - \sqrt{V^{2} + 4 \cdot V \cdot \mu + 4 \cdot \Delta^{2} + 4 \cdot \mu^{2}}} - 1 \right)^{2} \right)$"

In [97]:
LD = norm(reduced_density_matrix(vo * vo' - ve * ve', (1,), c))

|                                           2                                 
|                                        4*Δ                                  
|-----------------------------------------------------------------------------
|                                          /          ________________________
|/             __________________________\ |             _____________________
||            /  2              2      2 | |            /  2              2   
|\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 4*μ  /*\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 

          |
          |
------ - 1|
_____\    |
_____|    |
   2 |    |
4*μ  /    |

In [126]:
diagH = QuantumDots.DiagonalizedHamiltonian(vcat(diag(vals_o),diag(vals_e)),BlockDiagonal([Mo,Me]))

QuantumDots.DiagonalizedHamiltonian{Vector{Sym}, BlockDiagonal{Sym, Matrix{Sym}}}(Sym[-t + μ, t + μ, V/2 + μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2, V/2 + μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2], Sym[μ t 0 0; t μ 0 0; 0 0 0 -Δ; 0 0 -Δ V + 2*μ])

In [29]:
eigblocks = blocks(diagH; full=true)
ground_states = map(QuantumDots.ground_state, eigblocks)
ϕo = ground_states[1].vector
ϕe = ground_states[2].vector

16-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.683539171799759
  0.3003347769685026
 -0.2900394004762402
  0.3198021876132616
  0.2557644093310968
 -0.2900394004762404
  0.3003347769685026
  0.1282094542610353

In [52]:
δρ = ϕe*ϕe' - ϕo*ϕo'
LD = norm(reduced_density_matrix(δρ,(1,),c))

0.05936079886511918

In [111]:
coeffs = majorana_coefficients(ground_states...,c)
mpu = majorana_polarization(coeffs..., (1,)).mpu

0.7985341677281106

In [53]:
function solve(H)
    diagH = QuantumDots.diagonalize(H)
    eigblocks = blocks(diagH; full=true)
    ((Eo,ϕo), (Ee,ϕe)) = map(QuantumDots.ground_state, eigblocks)
    ρe = ϕe*ϕe'
    ρo = ϕo*ϕo'
    δρ = ϕo-ϕe
    gap = Eo-Ee
    w,z = majorana_coefficients(ϕo, ϕe, c)
    R = Tuple(1:div(N,2))
    mpu = majorana_polarization(w,z, R).mpu
    LD = reduced_density_matrix(δρ, R, c)
    return gap, mpu, LD
end
solve(μ,t,Δ,V) = solve(kitaev_hamiltonian(μ,t,Δ,V))

solve (generic function with 2 methods)

In [54]:
solve(1.0,1,2,1.0)

(0.49597691398647026, 0.7324678436633643, [1.048092283813159 0.2603920647636982 -0.20366826500294938 0.5302545115518197; 0.2603920647636982 0.28740508272532744 -0.32773471719152725 0.1112530439384502; -0.20366826500294938 -0.32773471719152725 0.3929888228890849 -0.07508449798197797; 0.5302545115518197 0.1112530439384502 -0.07508449798197797 0.2715138105724287])

In [55]:
function cost(μ,t) 
    gap, mpu, LD = solve(μ,t,1.0,.5)
    10^3*gap^2 + 1-mpu
end
cost(args) = cost(args...)

cost (generic function with 2 methods)

In [56]:
using BlackBoxOptim

In [57]:
opt = bboptimize(cost; NumDimensions = 2)

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps


0.50 secs, 3999 evals, 3886 steps, improv/step: 0.161 (last = 0.1611), fitness=0.000000004


1.00 secs, 8202 evals, 8091 steps, improv/step: 0.161 (last = 0.1605), fitness=0.000000004



Optimization stopped after 10001 steps and 1.23 seconds
Termination reason: Max number of steps (10000) reached
Steps per second = 8097.98
Function evals per second = 8187.85
Improvements/step = 0.13760
Total function evaluations = 10112


Best candidate found: [-0.205806, -0.754626]

Fitness: 0.000000004



BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (10000) reached", 10001, 1.694598612864e9, 1.2349998950958252, ParamsDictChain[ParamsDictChain[Dict{Symbol, Any}(:RngSeed => 688271, :NumDimensions => 2, :MaxSteps => 10000),Dict{Symbol, Any}()],Dict{Symbol, Any}(:CallbackInterval => -1.0, :TargetFitness => nothing, :TraceMode => :compact, :FitnessScheme => ScalarFitnessScheme{true}(), :MinDeltaFitnessTolerance => 1.0e-50, :NumDimensions => :NotSpecified, :FitnessTolerance => 1.0e-8, :TraceInterval => 0.5, :MaxStepsWithoutProgress => 10000, :MaxSteps => 10000…)], 10112, ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64, Vector{Float64}}(4.032772005224672e-9, [-0.20580625919641177, -0.754626089690799]), BlackBoxOptim.PopulationOptimizerOutput{FitPopulation{Float64}}(FitPopulation{Float64}([-0.2058064215810058 -0.20580561355895347 … -0.2058072355475354 -0.20580661252628726; -0.7546260754141363 -0.754626209129304 … -0.75462

In [102]:
best_candidate(opt)

2-element Vector{Float64}:
  0.43289240307001087
 -0.26754221874193773

### Transport

In [124]:
function get_leads(c, T, μ, Γ=1)
    N = QuantumDots.nbr_of_fermions(c)
    left = QuantumDots.NormalLead(Γ * c[1]'; T, μ=μ[1])
    right = QuantumDots.NormalLead(Γ * c[N]'; T, μ=μ[2])
    return (; left, right)
end
PauliSystem(H,leads) = QuantumDots.Pauli()(QuantumDots.OpenSystem(H, leads))

PauliSystem (generic function with 1 method)

In [134]:
@syms V1::real V2::real T::real
symleads = get_leads(c,T,(V1,V2));

In [140]:
paulisystem = QuantumDots.Pauli()(QuantumDots.diagonalize(QuantumDots.OpenSystem(diagH, symleads)));

In [143]:
lindbladsystem = QuantumDots.Lindblad()(QuantumDots.diagonalize(QuantumDots.OpenSystem(diagH, symleads)));

In [147]:
Matrix(lindbladsystem);

In [154]:
function conductance(H,T,V1,V2)
    # H = kitaev_hamiltonian(μ,t,Δ,V,T)
    leads = get_leads(c,T,(V1,V2))
    sys = QuantumDots.Pauli()(QuantumDots.diagonalize(QuantumDots.OpenSystem(H, leads)));
    conductance_matrix(sys)
end

conductance (generic function with 1 method)

In [155]:
conductance(ham(1.0,1,1,1), 1,0,0)

MethodError: MethodError: no method matching init(::SciMLBase.LinearProblem{Vector{Float64}, true, SciMLOperators.MatrixOperator{Float64, Matrix{Float64}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}}, Vector{Float64}, SciMLBase.NullParameters, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}})

Closest candidates are:
  init(!Matched::SciMLBase.OptimizationProblem, !Matched::Any, !Matched::Any...; kwargs...)
   @ SciMLBase C:\Users\svens\.julia\packages\SciMLBase\l4PVV\src\solve.jl:146


In [136]:
conductance_matrix(system)

MethodError: MethodError: no method matching init(::SciMLBase.LinearProblem{Vector{Sym}, true, SciMLOperators.MatrixOperator{Sym, Matrix{Sym}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}}, Vector{Sym}, SciMLBase.NullParameters, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}})

Closest candidates are:
  init(!Matched::SciMLBase.OptimizationProblem, !Matched::Any, !Matched::Any...; kwargs...)
   @ SciMLBase C:\Users\svens\.julia\packages\SciMLBase\l4PVV\src\solve.jl:146


In [125]:
PauliSystem(H,leads)

MethodError: MethodError: no method matching eigen!(::Hermitian{Sym, Matrix{Sym}})

Closest candidates are:
  eigen!(::Union{Hermitian{T, var"#s969"}, Hermitian{Complex{T}, var"#s969"}, Symmetric{T, var"#s969"}} where var"#s969"<:(StridedMatrix{T} where T), !Matched::AbstractMatrix{T}; sortby) where T<:Number
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\symmetriceigen.jl:167
  eigen!(!Matched::StridedMatrix{T}; permute, scale, sortby) where T<:Union{Float32, Float64}
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\eigen.jl:149
  eigen!(!Matched::StridedMatrix{T}; permute, scale, sortby) where T<:Union{ComplexF64, ComplexF32}
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\eigen.jl:172
  ...
